In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nibabel as nib
from tensorflow import keras
import tensorflow_addons as tfa

In [2]:
def data_gen():
    df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    for i in range(df.shape[0]):
        path = "../input/cervical-ct-preprocessing/" + str(df.iloc[i]["StudyInstanceUID"]) +".nii.gz"
        try:
            ct = nib.load(path)
            ct = ct.get_fdata()
            yield ({"input_1": ct}, {'co': df.iloc[i]["patient_overall"], 
                                    'c1': df.iloc[i]["C1"],
                                    'c2': df.iloc[i]["C2"],
                                    'c3': df.iloc[i]["C3"],
                                    'c4': df.iloc[i]["C4"],
                                    'c5': df.iloc[i]["C5"],
                                    'c6': df.iloc[i]["C6"],
                                    'c7': df.iloc[i]["C7"],})
        except:
            continue

In [3]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([100,100,60])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

2022-10-06 15:13:29.410627: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 15:13:29.425584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 15:13:29.426763: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 15:13:29.429721: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
dataset = dataset.batch(2)
dataset = dataset.shuffle(buffer_size = 20, seed=3)
dataset=dataset.prefetch(1)


In [5]:
inputA = keras.layers.Input(shape=(100,100,60), name="input_1")
X = keras.layers.Conv2D(1024, 3)(inputA)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(512, 3)(inputA)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(256, 3)(inputA)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(128, 3)(inputA)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(64, 3)(inputA)
X = keras.layers.MaxPooling2D()(X)

X = keras.layers.Flatten()(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Dense(1024 ,activation = 'relu')(X)
X = keras.layers.Dense(512 ,activation = 'relu')(X)
X = keras.layers.Dense(256 ,activation = 'relu')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Dense(128 ,activation = 'relu')(X)
X = keras.layers.Dense(16 ,activation = 'relu')(X)
outputo = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(X)
output1 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c1")(X)
output2 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c2")(X)
output3 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c3")(X)
output4 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c4")(X)
output5 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c5")(X)
output6 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c6")(X)
output7 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c7")(X)
                                               
model = keras.models.Model(inputs=inputA, outputs=[outputo,output1,output2,output3,output4,output5,output6,output7])

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 60 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 98, 98, 64)   34624       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 49, 49, 64)   0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
flatten (Flatten)               (None, 153664)       0           max_pooling2d_4[0][0]            
______________________________________________________________________________________________

In [7]:
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.0005,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
model.compile(optimizer=opt,
              loss={"co": loss, 
                   "c1": loss,
                   "c2": loss,
                   "c3": loss,
                   "c4": loss,
                   "c5": loss,
                   "c6": loss,
                   "c7": loss,},
                metrics={"co": metrics, 
                   "c1": metrics,
                   "c2": metrics,
                   "c3": metrics,
                   "c4": metrics,
                   "c5": metrics,
                   "c6": metrics,
                   "c7": metrics,})

In [ ]:
model.fit(dataset.take(700), validation_data = dataset.skip(700), epochs=10)

Epoch 1/10


2022-10-06 15:13:33.896786: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-06 15:13:35.799558: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


    700/Unknown - 50s 64ms/step - loss: 3.4319 - co_loss: 0.7396 - c1_loss: 0.3462 - c2_loss: 0.4796 - c3_loss: 0.2547 - c4_loss: 0.2783 - c5_loss: 0.3680 - c6_loss: 0.4478 - c7_loss: 0.5177 - c7_c6_c5_c4_c3_c2_c1_co_auc: 0.6856

Exception ignored in: <generator object data_gen at 0x7f9a4e018750>
RuntimeError: generator ignored GeneratorExit


700/700 [==============================] - 86s 114ms/step - loss: 3.4319 - co_loss: 0.7396 - c1_loss: 0.3462 - c2_loss: 0.4796 - c3_loss: 0.2547 - c4_loss: 0.2783 - c5_loss: 0.3680 - c6_loss: 0.4478 - c7_loss: 0.5177 - c7_c6_c5_c4_c3_c2_c1_co_auc: 0.6856 - val_loss: 3.7475 - val_co_loss: 0.7839 - val_c1_loss: 0.3340 - val_c2_loss: 0.4840 - val_c3_loss: 0.2567 - val_c4_loss: 0.3650 - val_c5_loss: 0.4633 - val_c6_loss: 0.4470 - val_c7_loss: 0.6135 - val_c7_c6_c5_c4_c3_c2_c1_co_auc: 0.7322
Epoch 2/10
700/700 [==============================] - 77s 109ms/step - loss: 2.9185 - co_loss: 0.6891 - c1_loss: 0.2682 - c2_loss: 0.4305 - c3_loss: 0.1456 - c4_loss: 0.2037 - c5_loss: 0.2746 - c6_loss: 0.4034 - c7_loss: 0.5033 - c7_c6_c5_c4_c3_c2_c1_co_auc: 0.7484 - val_loss: 3.8709 - val_co_loss: 0.8516 - val_c1_loss: 0.3556 - val_c2_loss: 0.5434 - val_c3_loss: 0.1559 - val_c4_loss: 0.2707 - val_c5_loss: 0.3070 - val_c6_loss: 0.6096 - val_c7_loss: 0.7772 - val_c7_c6_c5_c4_c3_c2_c1_co_auc: 0.6967


...

Epoch 5/10
700/700 [==============================] - 77s 109ms/step - loss: 2.8863 - co_loss: 0.6851 - c1_loss: 0.2616 - c2_loss: 0.4246 - c3_loss: 0.1405 - c4_loss: 0.1973 - c5_loss: 0.2801 - c6_loss: 0.3990 - c7_loss: 0.4980 - c7_c6_c5_c4_c3_c2_c1_co_auc: 0.7607 - val_loss: 3.9628 - val_co_loss: 0.8402 - val_c1_loss: 0.3326 - val_c2_loss: 0.4517 - val_c3_loss: 0.2373 - val_c4_loss: 0.2822 - val_c5_loss: 0.7921 - val_c6_loss: 0.3895 - val_c7_loss: 0.6372 - val_c7_c6_c5_c4_c3_c2_c1_co_auc: 0.7381
Epoch 6/10


Exception ignored in: <generator object data_gen at 0x7f9a0e2aa550>
RuntimeError: generator ignored GeneratorExit


700/700 [==============================] - 77s 109ms/step - loss: 2.8808 - co_loss: 0.6844 - c1_loss: 0.2615 - c2_loss: 0.4213 - c3_loss: 0.1370 - c4_loss: 0.1991 - c5_loss: 0.2826 - c6_loss: 0.3990 - c7_loss: 0.4959 - c7_c6_c5_c4_c3_c2_c1_co_auc: 0.7643 - val_loss: 5.5683 - val_co_loss: 0.9410 - val_c1_loss: 0.4713 - val_c2_loss: 0.4856 - val_c3_loss: 0.2647 - val_c4_loss: 0.2566 - val_c5_loss: 0.5751 - val_c6_loss: 0.7899 - val_c7_loss: 1.7843 - val_c7_c6_c5_c4_c3_c2_c1_co_auc: 0.7081
Epoch 7/10


Exception ignored in: <generator object data_gen at 0x7f9a4d0cddd0>
RuntimeError: generator ignored GeneratorExit


496/700 [====================>.........] - ETA: 12s - loss: 2.8241 - co_loss: 0.6918 - c1_loss: 0.2446 - c2_loss: 0.4207 - c3_loss: 0.1300 - c4_loss: 0.1814 - c5_loss: 0.2588 - c6_loss: 0.3867 - c7_loss: 0.5101 - c7_c6_c5_c4_c3_c2_c1_co_auc: 0.7674